In [12]:
import pandas as pd
from fastai.tabular.all import *

# Load your already cleaned/train DataFrame (use what you had with SalePrice, Id, etc)
df = pd.read_csv("../../data/train.csv")

# Optionally: use your earlier data cleaning code here to match your best results!

# Set which columns are categorical and which are continuous:
cat_names = df.select_dtypes("object").columns.tolist()
cont_names = [c for c in df.columns if c not in cat_names + ["SalePrice", "Id"]]

# We'll predict the log of SalePrice
df["LogSalePrice"] = np.log1p(df["SalePrice"])

In [13]:
cat_names = df.select_dtypes("object").columns.tolist()
cont_names = df.select_dtypes(["float64", "int64"]).columns.tolist()
cont_names = [c for c in cont_names if c not in ["SalePrice", "Id"]]

In [22]:
# Assume your train pipeline looks like this:
to = TabularPandas(
    train_df,  # Your full train data with target
    procs=[Categorify, FillMissing, Normalize],
    cat_names=cat_names,
    cont_names=cont_names,
    y_names="LogSalePrice",
    splits=splits,
)
dls = to.dataloaders(bs=64)
learn = tabular_learner(dls, ...)

# Now for test set prediction:
test_df = pd.read_csv("../data/test.csv")

# 1. This is the correct way:
test_dl = dls.test_dl(test_df)  # <-- NO TabularPandas here!
preds, _ = learn.get_preds(dl=test_dl)

# 2. Prepare for Kaggle submission
submission = pd.DataFrame(
    {
        "Id": test_df["Id"],
        "SalePrice": np.expm1(
            preds.numpy().squeeze()
        ),  # use expm1 if you trained on log
    }
)
submission.to_csv("submission.csv", index=False)

NameError: name 'train_df' is not defined